# Week 8
This week's practice problems cover:
* Containers: Tuples and Sets, Looping, Indexing, and Slicing
* Dictionaries



## Question 1
Ask the user for a size (`size`) and a maximum value (`max_value`). Write a program that randomly assigns values between 0 and `max_value` to a `size`-by-`size` matrix. **The matrix should be represetned as a nested tuple (i.e. a tuple of tuples).** Print the matrix to the screen, and then calculate the sum of values around the "outside" (border) of the matrix: that is, the sum of the entries in the first and last rows and first and last columns but without double counting the corner entries. For example, if the random matrix is:

    8 7 2 0 4
    8 4 0 2 5
    1 9 4 6 5
    0 2 6 9 3
    1 7 3 9 5

then the sum around the outside is 68. You will need to import the random module and use an appropriate function for generating the numbers.




In [7]:
import random
import pprint

# A couple of solutions. I prefer sum_with_sum

def sum_with_sum(matrix):
    ''' (2D-list) -> int
    Returns the sum of the elements in the border of the size X size matrix. 
    First and last rows and first and last columns.
    '''
    size = len(matrix)
    s = sum(matrix[0]) # first row
    
    for x in range(1,size - 1):
        # first and last elements of middle rows
        s += matrix[x][0] + matrix[x][size - 1] 
        
    s += sum(matrix[size - 1]) # last row

    return s

def sum_with_loops(matrix):
    ''' (2D-list) -> int
    Returns the sum of the elements in the border of the size X size matrix. 
    First and last rows and columns.
    '''
    size = len(matrix)
    s = 0
    for row in range(size):
        if row == 0 or row == size - 1:
            # sum whole row for first and last rows
            for value in matrix[row]:
                s += value
        else:
            # sum first and last entries for middle rows
            s += matrix[row][0]
            s += matrix[row][size - 1]
            
    return s

size = int(input("Please input the matrix size (e.g., 5): "))
max_value = int(input("Please input the maximum value: "))

# create the matrix
matrix = ()
for x in range(size):
    row = ()
    for y in range(size):
        row += (random.randint(0, max_value), )
    
    matrix += (row, )

pprint.pp(matrix) # Bonus question: print out each row of matrix on a different line

# sum the borders
print(sum_with_sum(matrix))
print(sum_with_loops(matrix))

Please input the matrix size (e.g., 5):  5
Please input the maximum value:  3


((1, 3, 3, 1, 3),
 (1, 1, 1, 0, 2),
 (0, 0, 3, 1, 0),
 (3, 1, 1, 2, 3),
 (1, 2, 0, 2, 2))
27
27


## Question 2
Write a program that asks the user for a matrix size (call it, `n`) and then randomly assigns the values 1 to `n`<sup>2</sup> to a `n` x `n` matrix, **not repeating any values!** Again, the matrix should be represetned as a nested tuple.

Then print the matrix to the screen.  For example:

      3 25 12  4 19
      5 17 21 10  6
      1  8 16  2  7
     13 22 24 11 20
     23 14 15  9 18 

Notice that you'll need some way to make sure there are no repeated numbers. 

If you do this in a simple, but inefficient way, you’ll notice it takes longer and longer to place the next value or find the next location. That’s OK for a 5 x 5 matrix but won’t be appropriate for a much larger (say, 1000 x 1000) matrix. So then what? Try writing a version that’s efficient, that takes the same amount of time to place the first number as the last. Maybe you can think of more than two different ways?

Bonus: Investigate the Python `time` module and time your different implementations.


In [71]:
import random
import timeit

# a helper function
def create_zero_matrix(size):
    ''' (int) -> 2D tuple
    Returns an size X size matrix initialized to 0
    '''
    # There is a more elegant way to do this with list comprehension but
    # that is a bit beyond this course
    matrix = ()
    for x in range(size):
        row = ()
        for y in range(size):
            row += (0, )
        matrix += (row,)
        
    return matrix

# A checking function that will help in debugging
def check_unique(matrix):
    ''' (tuple) -> bool
    Returns True if the size X size matrix contains only unique numbers
    from 1 to size**2
    '''
    size = len(matrix)
    count = []
    for i in range(size**2):
        count.append(0)
    
    for x in range(size):
        for y in range(size):
            index = matrix[x][y] - 1
            if count[index] == 0:
                count[index] += 1
            else:
                return False

    return True    

# Randomly generate each number and check if it has already been generated before
# If so, generate it again
def create_unique_matrix_gen(size):
    ''' (int) -> 2D tuple
    Returns a size X size matrix containing the numbers 1 to size**2 in random
    order.
    '''

    # start by filling a list with unique numbers in random order
    numbers = ()
    while len(numbers) < size**2:
        rnd_num = random.randint(1,size**2)
        while rnd_num in numbers:
            rnd_num = random.randint(1,size**2)
        numbers += (rnd_num, )
    
    # transfer list elements to matrix
    matrix = ()    
    start = 0
    end = size
    index = 0  
    while end <= size**2:
        matrix += (numbers[start:end], )
        start = end
        end += size
            
    return matrix

# A bit faster implementation than the above
def create_unique_matrix_gen1(size):
    ''' (int) -> 2D tuple
    Returns a size X size matrix containing the numbers 1 to size**2 in random
    order.
    '''

    # start by filling a list with unique numbers in sorted order
    ordered_numbers = list(range(1, size**2 + 1))
    numbers = ()
    while len(ordered_numbers) > 0:
        rnd_idx = random.randint(0, len(ordered_numbers) - 1 )
        rnd_num = ordered_numbers.pop(rnd_idx)
        numbers += (rnd_num, )
    
    # transfer list elements to matrix
    matrix = ()    
    start = 0
    end = size
    index = 0  
    while end <= size**2:
        matrix += (numbers[start:end], )
        start = end
        end += size
            
    return matrix

# Create a matrix of zeros. For each number randomly generate a cell and put
# it there if the cell is equal to 0. If not, generate a new cell
def create_unique_matrix_index(size):
    ''' (int) -> 2Dlist
    Returns a size X size matrix containing the numbers 1 to size**2 in random
    order.
    '''
    matrix = create_zero_matrix(size)
    num = 1
    while num <= size**2:
        # generate a random cell in the matrix
        i = random.randint(0, size - 1)
        j = random.randint(0, size - 1)
        
        # if the cell is empty, replace it with the number we want
        if matrix[i][j] == 0:            
            matrix = matrix[:i] + \
                     (matrix[i][:j] + (num,) + matrix[i][j+1:] ,)+ \
                     matrix[i+1:]  # Since tuple is immutable, we have to slice the tuple 
                                   # in order to replace a number within
            num += 1
    
    return matrix

# Generate a ordered list and for each element swap it with a randomly chosen
# element
def create_unique_matrix_swap(size):
    ''' (int) -> 2D tuple
    Returns a size X size matrix containing the numbers 1 to size**2 in random
    order.
    '''
    one_d = list(range(size**2)) # create a list of 0 to size**2 - 1
                                 # To do swap, it is much easier to use a list here because it is immutable
    for x in range(len(one_d)):
        # pick random index and swap
        rnd_index = random.randint(0,len(one_d) - 1)
        (one_d[x], one_d[rnd_index]) = (one_d[rnd_index], one_d[x])
    #print(one_d)
    
    # create matrix from one_d list
    index = 0   
    matrix = ()
    for x in range(size):
        row = ()
        for y in range(size):
            row += (one_d[index] + 1, ) # add one since one_d has numbers from 0
            index += 1
        matrix += (row, )
                    
    return matrix


size = int(input("Matrix size (e.g., 5): "))

n = int(input("How many times do you want to test the code (e.g., 10000): "))

# Test the slow version
print("Testing create_unique_matrix_gen")

start_time = timeit.default_timer()

for i in range(n):
    if i % 100 == 0:
        print("Test#: ", i,end = "\r")

    matrix = create_unique_matrix_gen(size)
    if not check_unique(matrix):
        print("\nError!!")
        print(matrix)
    assert check_unique(matrix)
        
print("\nCumulative time: ", timeit.default_timer() - start_time)



print("Testing create_unique_matrix_gen1")

start_time = timeit.default_timer()

for i in range(n):
    if i % 100 == 0:
        print("Test#: ", i,end = "\r")

    matrix = create_unique_matrix_gen1(size)
    if not check_unique(matrix):
        print("\nError!!")
        print(matrix)
    assert check_unique(matrix)
        
print("\nCumulative time: ", timeit.default_timer() - start_time)

# Test the slower version
print("\nTesting create_unique_matrix_index")

start_time = timeit.default_timer()

for i in range(n):
    if i % 100 == 0:
        print("Test#: ", i,end = "\r")

    matrix = create_unique_matrix_index(size)
    if not check_unique(matrix):
        print("\nError!!")
        print(matrix)
    assert check_unique(matrix)

print("\nCumulative time: ", timeit.default_timer() - start_time)

# Test the faster version
print("\nTesting create_unique_matrix_swap")

start_time = timeit.default_timer()

for i in range(n):
    if i % 100 == 0:
        print("Test#: ", i,end = "\r")

    matrix = create_unique_matrix_swap(size)
    
    if not check_unique(matrix):
        print("\nError!!")
        print(matrix)
    assert check_unique(matrix)
        
print("\nCumulative time: ", timeit.default_timer() - start_time)

Matrix size (e.g., 5):  6
How many times do you want to test the code (e.g., 10000):  10000


Testing create_unique_matrix_gen
Test#:  9900
Cumulative time:  2.0366215887479484
Testing create_unique_matrix_gen1
Test#:  9900
Cumulative time:  0.574348445981741

Testing create_unique_matrix_index
Test#:  9900
Cumulative time:  2.909360473975539

Testing create_unique_matrix_swap
Test#:  9900
Cumulative time:  0.5907814237289131


## Question 3

Assume an array of N elements represented as a sorted tuple of integers.  Write a function that corresponds to the following definition:

```python
def binary_search (array, value)
```

The function does a binary search, and returns the index in the tuple where value is stored.  If value isn’t in the tuple, the function returns -1.

A binary search is a way to search a sorted array of numbers.  Begin at the middle of the array and decide if the value you're looking for is in the first half of the array or the second half.  If it is the first half, then look at the middle element of the first half (i.e., a quarter of the way through the initial array) and decide if the element is in the first quarter or second quarter. If it is in the second half, look at the middle element of the second half (i.e. three-quarters of the way through the original array) to decide if you should like in the third quarter or fourth quarter. Then look halfway again, and halfway again, and … 

Test your function on an array of size 100 filled in with sorted integers.

Hint: Draw a picture of what the binary search is supposed to do and while you are writing the code, use a short list (e.g., less than 10 elements) to debug it.




In [65]:
import random

def binary_search(numbers, num):
    ''' (tuple of int, int) -> int
    Returns the index of num in numbers or -1 if num is not in the list
    numbers is a sorted list
    '''
    min_index = 0
    max_index = len(numbers) - 1
    while(min_index <= max_index):
        #print(min_index, max_index)
        
        middle = (min_index + max_index)//2
        
        if numbers[middle] > num:
            max_index = middle - 1
        elif numbers[middle] < num:
            min_index = middle + 1
        else:
            return middle

    return -1
    
size = int(input("How large a tuple: "))

# create a sorted list of random numbers
numbers = []  # To leverage the sort() method of a list, just use a list instead
              # and convert it to tuple later as requested
for i in range(size):
    numbers.append(random.randint(0, 10000))

numbers.sort()
numbers = tuple(numbers)  # This is purly optional as the binary search function
                          # can take in either a list or a tuple as the input
print(numbers)

# choose a random index and search for the corresponding value
rand_index = random.randint(0,size-1)
index = binary_search(numbers,numbers[rand_index]) 

if rand_index == index:
    print("Found it! rand_index:", rand_index, "\t index:", index,"\tvalue: ", numbers[rand_index])
else:
    print("*** Problem! rand_index:", rand_index, "\t index:", index)

# Test for case when the value isn't there
num = numbers[rand_index]
print("Removing", num)
numbers = numbers[:rand_index]  + numbers[rand_index+1:]  # This is how you would do to remove a number from a tuple

index = binary_search(numbers,num) 
if index == -1:
    print(num,"is no longer in the list")
else:
    print("*** Problem!", num, "is still there. index: ", index)

How large a tuple:  111


(76, 321, 338, 442, 550, 579, 649, 651, 724, 773, 806, 830, 853, 862, 921, 1076, 1162, 1319, 1359, 1382, 1385, 1396, 1742, 1860, 1976, 2055, 2105, 2151, 2158, 2375, 2399, 2704, 2728, 2735, 2796, 2983, 2986, 3008, 3082, 3088, 3117, 3663, 3721, 3907, 4059, 4151, 4222, 4318, 4378, 4390, 4440, 4477, 4521, 4598, 4738, 4793, 4807, 4819, 4834, 4863, 5135, 5139, 5233, 5414, 5456, 5486, 5497, 5516, 5645, 5734, 5920, 5973, 6174, 6265, 6608, 6728, 6823, 6902, 6927, 6956, 6985, 7127, 7129, 7230, 7302, 7315, 7339, 7347, 7927, 8081, 8229, 8266, 8327, 8410, 8546, 8757, 8820, 8884, 8986, 8987, 9006, 9200, 9290, 9304, 9425, 9668, 9746, 9782, 9822, 9962, 9984)
Found it! rand_index: 34 	 index: 34 	value:  2796
Removing 2796
2796 is no longer in the list


## Question 4

Write a function called `find_dups` that takes a list of integers as its input argument and returns a set of those integers occurring two or more times in the list. (Question source: Gries et al., Practical Programming, 3rd Edition).

In [66]:

def find_dups(L):
    """ (list) -> set

    Return a set containing the duplicated elements in L.

    >>> find_dups([1, 1, 2, 3, 4, 2])
    {1, 2}
    >>> find_dups([1, 2, 3, 4])
    set()
    """ 

    elem_set = set()
    dups_set = set()

    # for each entry in the list, if it doesn't make the set of elements bigger
    # then it is a duplicate
    for entry in L:
        len_initial = len(elem_set)
        elem_set.add(entry)
        len_after = len(elem_set)
        if len_initial == len_after:
            dups_set.add(entry)

    return dups_set

print(find_dups([1,2,3,3,4,19,0,1]))
print(find_dups(['Smith', 'O\'Brien', 'Smith', 'Mansour', 'Bryant', 'Smith']))

{1, 3}
{'Smith'}


## Question 5
Generalize the function from Q4 to `find_multiples`. The function should take an additional parameter, `k`, and return a set of those elements that appear `k` or more times. Note: depending on how you answered Q4, this function may require a quite different approach. 

You should implement the function it in two ways: using one of the lists methods and using a dictionary. Your code should work with a list of any type.


In [67]:
def find_multiples(L, k):
    """ (list) -> set
    Return the a set containing the elements of L that occur k or more times.
    """ 

    dups_set = set()

    # for each entry in the list, if it appears k or more times
    # then add it to the dups_set
    for entry in L:
        if L.count(entry) >= k:
            dups_set.add(entry)

    return dups_set


# An alternative implementation using a dictionary
def find_multiples_dict(L, k):
    """ (list) -> set
    Return the a set containing the elements of L that occur k or more times.
    """ 

    dups_set = set()
    counts = {}
    
    # for each entry in the list, count its occurences and add to the
    # dups if it occurs k times
    for entry in L:
        if entry in counts:
            counts[entry] += 1
        else:
            counts[entry] = 1

        if counts[entry] == k:   # Bonus question: why is this == and not >=?
            dups_set.add(entry)

    return dups_set

print(find_multiples([1,2,3,3,4,19,0,1,3,0,0], 3))
print(find_multiples(['Smith', 'O\'Brien', 'Smith', 'Mansour', 'Bryant', 'Smith'], 2))

print(find_multiples_dict([1,2,3,3,4,19,0,1,3,0,0], 3))
print(find_multiples_dict(['Smith', 'O\'Brien', 'Smith', 'Mansour', 'Bryant', 'Smith'], 2))

{0, 3}
{'Smith'}
{0, 3}
{'Smith'}


## Question 6

1. Write a function that takes in two equal size sets with arbitrary element types and returns a set of element pairs containing each element from the input sets in exactly one pair. For example, input `{'a','b'}` and `{1,2}` could produce output `{('a',1), ('b', 2)}`.  [A variation on a question from Gries et al., Practical Programming, 3rd Edition].
2. Write a variation on the function in part a that returns a much larger set with each element in the first set matched with each element in the second set. The two input sets no longer need to be the same size.


In [68]:
def generate_corresponding_pairs(s1, s2):
    """ 
    (set, set) -> set of tuple

    Return a set of tuples where each tuple contains an entry from s1
    and an entry from s2.
    """ 

    pairs = set()
    n = len(s1)

    for i in range(n):

        entry1 = s1.pop()
        entry2 = s2.pop()
        pairs.add((entry1, entry2),)

    return pairs

def generate_all_pairs(s1, s2):
    """ 
    (set, set) -> set of tuple

    Return a set of tuples where each tuple contains an entry from s1
    and an entry from s2.
    """ 

    pairs = set()

    for i in s1:
        for j in s2:
            pairs.add((i, j),)

    return pairs

print(generate_corresponding_pairs({'Anne', 'Beatrice', 'Cari'}, 
                                   {'Ali', 'Bob', 'Chen'}))
print(generate_all_pairs({'Anne', 'Beatrice', 'Cari'}, 
                                   {'Ali', 'Bob', 'Chen'}))

{('Beatrice', 'Bob'), ('Anne', 'Chen'), ('Cari', 'Ali')}
{('Beatrice', 'Chen'), ('Anne', 'Chen'), ('Cari', 'Bob'), ('Cari', 'Chen'), ('Beatrice', 'Ali'), ('Cari', 'Ali'), ('Anne', 'Ali'), ('Beatrice', 'Bob'), ('Anne', 'Bob')}


## Question 7

[Question source: Gries et al,. Practical Programming, 3rd Edition] 
A sparse vector is a one whose entries are almost all zero, e.g., `[1, 0, 0, 0, 0, 0, 3, 0, 0, 0]`. To save memory, such sparse vectors can be stored as a dictionary that just keeps the non-zero entries. For example, the vector above could be represented as `{0:1, 6:3}`: the value 1 at index 0 and the value 3 at index 6.
1. Write a function called `sparsify` that takes in a list of integers and returns a dictionary representing a sparse vector.
2. The sum of two vectors is the element-wise sum of their elements. For example, the sum of `[9, 9, 9]` and `[1, 2, 3]` is `[10, 11, 12]`. Write a function called `sparse_add` that takes two sparse vectors stored as dictionaries and returns a new dictionary representing their sum.
3. The dot product of two vectors is the sum of the products of corresponding elements. For example, the dot product of `[1, 2, 3]` and `[4, 5, 6]` is 4+10+18, or 32. Write a function called sparse_dot that calculates the dot product of two sparse vectors.
4. Run the following code and observe the output:
```python
sparse_add(sparsify([0,0,1,-2,0,0])
sparsify([0,1,0,2,0,0])
```
Depending on how you implemented the method above, the resulted sparse vector may end up containing an entry for a zero element, because -2 + 2 = 0. To fix this problem, let's write a function that takes in a sparse vector and remove zero element entries, if any.
5. As an alternative to 4, re-write your `sparse_add` function to always return a valid sparse vector (i.e. does not contain an entry for a zero element).


In [69]:
# Q12 from p. 228 Gries book 3rd edition

    
def sparse_add(vector1, vector2):
    """ (dict of {int: int}, dict of {int: int} -> dict of {int: int})

    Return the sum of sparse vectors vector1 and vector2.

    >>> sparse_add({1: 3, 3: 4}, {2: 4, 3: 5, 5: 6})
    {1: 3, 2: 4, 3: 9, 5: 6}
    """ 

    sum_vector = vector1.copy()

    # if there is an element in both, add them
    # if the element is only in vector2, insert it into the sum_vector
    for key in vector2:
        if key in sum_vector:
            sum_vector[key] = sum_vector[key] + vector2[key]
        else:
            sum_vector[key] = vector2[key]

    return sum_vector


def sparse_dot(vector1, vector2):
    """ (dict of {int: int}, dict of {int: int} -> dict of {int: int})

    Return the dot product of sparse vectors vector1 and vector2.

    >>> sparse_dot({1: 3, 3: 4}, {2: 4, 3: 5, 5: 6})
    20
    """ 

    dot = 0
    # if there is an entry in both, multiply
    # if not entry in both, ignore (since multiplication will be 0)
    for key1 in vector1:
        if key1 in vector2:
            dot += vector1[key1] * vector2[key1]

    return dot

# Q7c: Create a function that takes in a list of integers and returns the dictionary
# representing the sparse vector
def sparsify(list):
    ''' (list of int) -> dict of {int: int}
    Returns the sparse vector version of list
    '''
    vec = {}
    # add all non-zero entries to the sparse vector
    for i in range(len(list)):
        if list[i] != 0:
            vec[i] = list[i]
            
    return vec

# Q7d: Create a function that takes in a sparse vector dictionary and re-sparsifies it
def re_sparsify(sparse_vec):
    ''' (dict of {int: int}) -> dict of {int: int}
    Returns sparse_vec with any 0 entries removed
    '''
    vec = {}
    # add all non-zero entries to the new sparse vector
    for k, v in sparse_vec.items():
        if v != 0:
            vec[k] = v
            
    return vec

# Q7e: Create a new add function that always returns a valid sparse vector
def sparse_add_fixed(vector1, vector2):
    """ (dict of {int: int}, dict of {int: int} -> dict of {int: int})

    Return the sum of sparse vectors vector1 and vector2.

    >>> sparse_add({1: 3, 3: 4}, {2: 4, 3: 5, 5: 6})
    {1: 3, 2: 4, 3: 9, 5: 6}
    """ 

    sum_vector = vector1.copy()

    # if there is an element in both, add them
    # if the element is only in vector2, insert it into the sum_vector
    for key in vector2:
        if key in sum_vector:
            sum_vector[key] = sum_vector[key] + vector2[key]
            if sum_vector[key] == 0:
                sum_vector.pop(key)
        else:
            sum_vector[key] = vector2[key]

    return sum_vector

v = sparsify([1,0,0,0,0,0,3,0,0,0])
sum_vec = sparse_add(v,v)
print(v)
print(sum_vec)

sub_vec = sparsify([1,0,0,0,0,0,-6,0,0,0])
sum2_vec = sparse_add(sum_vec, sub_vec)
re_sparse = re_sparsify(sum2_vec)

print(sub_vec)
print(sum2_vec)
print(re_sparse)

print(sparse_add(sparsify([0,0,1,-2,0,0]), sparsify([0,1,0,2,0,0])))

print(re_sparsify(sparse_add(sparsify([0,0,1,-2,0,0]), sparsify([0,1,0,2,0,0]))))

print(sparse_add_fixed(sparsify([0,0,1,-2,0,0]), sparsify([0,1,0,2,0,0])))

{0: 1, 6: 3}
{0: 2, 6: 6}
{0: 1, 6: -6}
{0: 3, 6: 0}
{0: 3}
{2: 1, 3: 0, 1: 1}
{2: 1, 1: 1}
{2: 1, 1: 1}


## Question 8

Write a program that asks a user to enter a string. Output letter that appears the most times and the number of times it appears. You need to deal with both capital and lower-case letters.

For example:

    Enter a word:  Bubble
    The letter b/B appears most often: 3 times.

Hint: Do it once using a list and then again using a dictionary. Don't look at Bonus #2 yet as it gives a big hint.

Bonus #1: Did you use the string `count()` function? This function reads through the entire string each time it is called. Since you need to count the number of occurrences of each letter in the word (let's say there are n letters), that means count will make n "steps"  n times (i.e., you will look for each letter (n steps) once for each letter (n letters)). So your program will execute n2 steps. See if you can write the solution without using `count()` such that you only look at each letter in the word once (or twice) not n times. 

Bonus #2: The simplest solutions will use two loops (once through the word to count the letters and once through the dictionary (or list) to find the one that appears most). So you will look at each letter twice, once per loop. Can you re-do the solution using a dictionary with only one loop?


In [70]:
# Write a program that asks a user to enter a string and output letter 
# that appears the most times and the number of times it appears. 
# You need to deal with both capital and lower-case letters.

def max_count_list(s):
    '''
    (str) -> tuple(str, int)
    Return the letter than appears the most times in s together with 
    the number of times it appears
    '''
    # initialize a list to store the number of times each letter appears
    for i in range(26):
        counts.append(0)

    # for each letter, add 1 to the list for that number
    for i in word:
        counts[ord(i) - ord('a')] += 1 # why does this work?

    # find the index of the max entry in a
    index = 0
    for i in range(len(counts)):
        if counts[i] > counts[index]:
            index = i

    return (chr(index+ord('a')), counts[index])

def max_count_dict(s):
    '''
    (str) -> tuple(str, int)
    Return the letter than appears the most times in s together with 
    the number of times it appears
    '''
    counts = {}

    # for each letter, add 1 to the list for that number
    for i in word:
        counts[i] = counts.get(i,0) + 1 # why does this work?

    # find the index of the max entry in a
    max_letter = word[0] # initialize to some letter that we know is in the dictionary
    for i in counts:
        if counts[i] > counts[max_letter]:
            max_letter = i

    return (max_letter, counts[max_letter])

def max_count_dict_one_loop(s):
    '''
    (str) -> tuple(str, int)
    Return the letter than appears the most times in s together with 
    the number of times it appears
    '''
    counts = {}

    max_letter = word[0] # initialize to some letter that we know is in the dictionary

   # for each letter, add 1 to the list for that number
    for i in word:
        counts[i] = counts.get(i,0) + 1 # why does this work?

        if counts[i] > counts[max_letter]:
            max_letter = i

    return (max_letter, counts[max_letter])

word = input("Enter a word: ")
counts = []
word = word.lower()

(letter, num_times) = max_count_list(word)
print("The letter " + letter +"/"+ letter.upper()+ " appears most often: ", num_times, "times." )

(letter, num_times) = max_count_dict(word)
print("The letter " + letter +"/"+ letter.upper()+ " appears most often: ", num_times, "times." )

(letter, num_times) = max_count_dict_one_loop(word)
print("The letter " + letter +"/"+ letter.upper()+ " appears most often: ", num_times, "times." )

Enter a word:  Apple


The letter p/P appears most often:  2 times.
The letter p/P appears most often:  2 times.
The letter p/P appears most often:  2 times.
